In [ ]:
import sys

import numpy as np

from opts import get_opts
from utils import get_data_base_path, make_figure, notebook_config

sys.path.insert(1, "/Users/user/Documents/Git_repos/climate-epidemics")

import climepi  # noqa
from climepi import climdata, epimod

# Configure notebook
notebook_config()

# Get options
options = get_opts()
year_range = options["year_range"]
location = options["location_default"]
epi_model_name = options["epi_model_name_default"]
polyfit_degree = options["polyfit_degree"]
plot_opts_temp = options["plot_opts_temp"]
plot_opts_epi = options["plot_opts_epi"]
figure_dir = options["figure_dir"]

In [2]:
# Get climate data
ds_clim = (
    climdata.get_example_dataset(
        "isimip_cities", data_dir=get_data_base_path() / "isimip_cities"
    )
    .sel(location=location)
    .sel(time=slice(str(year_range[0]), str(year_range[1])))
)
# Yearly average temperature data
ds_temp_yearly = ds_clim.climepi.yearly_average("temperature")
ds_temp_yearly["time"] = ds_temp_yearly["time.year"]  # avoids plotting issues

In [3]:
# Get and run epidemiological model
epi_model = epimod.get_example_model(epi_model_name)
ds_months_suitable = ds_clim.climepi.run_epi_model(
    epi_model, return_months_suitable=True
)
ds_months_suitable["time"] = ds_months_suitable["time.year"]  # avoids plotting issues

In [ ]:
# Climate variability - variance decomposition
p_clim_var = ds_temp_yearly.climepi.plot_var_decomp(polyfit_degree=polyfit_degree).opts(
    **{
        **plot_opts_temp,
        "ylim": (0, 5.1),
        "yticks": list(range(6)),
        "ylabel": "Variance of mean temperature (°C²)",
    }
)
p_clim_var

In [ ]:
# Epi variability - variance decomposition
p_epi_var = ds_months_suitable.climepi.plot_var_decomp(
    polyfit_degree=polyfit_degree
).opts(
    **{
        **plot_opts_epi,
        "ylim": (0, 2),
        "yticks": np.arange(0, 2.5, 0.5),
        "ylabel": "Variance of months suitable",
    }
)
p_epi_var

In [12]:
# Combine panels
panels = [p_clim_var, p_epi_var]
make_figure(
    figure_dir / "variance_plots.svg",
    panels,
    show_legend_list=[True, False],
)